## Extractor MongoDB a .JSON

#### Import Libraries

In [ ]:
from sshtunnel import SSHTunnelForwarder
import pymongo
import pprint
import json
from bson import json_util
import time
import threading
import warnings
warnings.filterwarnings("ignore")

#### Extractor 

In [ ]:
# Print time before before extract data
time1 = time.asctime()
print(time1)

# Credentials of MongoDB Prod
MONGO_HOST = "IP"
MONGO_USER = "USER"
MONGO_PASS = "PASSWORD"
PEM = "D:/Topic Flower/db"
MONGO_DB = "topicflower_prod"
MONGO_COLLECTION = "COLLECTION"

# Define local path
path = r'C:\Users\Darwoft\PostClassification\PostClasification\topics\\'


def extraer_coleccion(cole):
    collection = db[cole]

    # NoSql query to MongoDB to extract the first 100.000 post
    cursor = collection.find({"$and": [{'topic': {"$ne": None}}, {"$or": [{'twitter': {"$ne": None}}, {'data_facebook': {"$ne": None}}]}]}, {
                             "brandname": 1, "topic": 1, "topicname": 1, "twitter.posttext": 1, "data_facebook.message": 1}).limit(100000)
    texto = '['
    for fut in cursor:
        texto = texto + json.dumps(fut,  default=json_util.default) + ','
    texto = texto[:-1]
    texto = texto + ']'
    archivo_post = path + str(cole)+'.json'

    # Save the query data into a .json file
    with open(archivo_post, 'w', encoding='UTF-8') as file:
        file.write(texto)
    return None


# Define SSH Tunnel
server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_pkey=PEM,
    remote_bind_address=('127.0.0.1', 27017)
)

# Start SSH tunnel
server.start()

# Server.local_bind_port is assigned local port
client = pymongo.MongoClient(
    '127.0.0.1', server.local_bind_port, maxPoolSize=50, waitQueueMultiple=10)
db = client[MONGO_DB]

# call Function to extract Post collection data
extraer_coleccion('post')

# Close connection
client.close()
server.stop()
time2 = time.asctime()

# Print time for the whole process
print(time2)

## Convert .Json to CSV file

In [ ]:
"""
Before convert the .json to CSV file we need to find and replace the "data_facebook":¨{}
because to make it work we need to replace it for "data_facebook": {"message": ""}
If not we will have a error before convert to CSV file
This will be update in the future 
"""

#### Import Libraries

In [ ]:
import pandas as pd
import json
import re
import warnings
warnings.filterwarnings("ignore")

#### Convertor

In [ ]:
# Define Json and CSV files
archivo_post = r'C:\Users\Darwoft\PostClassification\PostClasification\topics\post.json'
archivo_sent = r'C:\Users\Darwoft\PostClassification\PostClasification\topics\Topics-text.csv'

# Open Json file
with open(archivo_post, 'r', encoding='UTF-8') as file:
    data = json.load(file)


facebooktext = ''
twittertext = ''

# Define CSV columns
texto = "id,brandname,topic,topicname,twittertext,facebooktext" + '\n'

for f in data:
    try:
        ID = f.get('_id').get('$oid')
        brandname = f.get('brandname')
        topic = f.get('topic')
        topic = topic.lower()
        topicname = f.get('topicname')
        twittertext = f.get('twitter').get('posttext')

        # Delete special characters, URL and Tags from the post
        twittertext = twittertext.replace(',', '')
        twittertext = twittertext.replace('\n', '')
        twittertext = twittertext.replace('\"', '')
        twittertext = twittertext.replace('"', '')
        twittertext = re.sub(r"http\S+", '', twittertext)
        twittertext = re.sub(r"@\S+", '', twittertext)
    except:
        facebooktext = f.get('data_facebook').get('message')

        # Delete special characters, URL and Tags from the post
        facebooktext = facebooktext.replace(',', '')
        facebooktext = facebooktext.replace('\n', '')
        facebooktext = facebooktext.replace('\"', '')
        facebooktext = facebooktext.replace('"', '')
        facebooktext = re.sub(r"http\S+", '', facebooktext)
        facebooktext = re.sub(r"@\S+", '', facebooktext)

    # Create CSV estruture separate by ","
    texto = texto + str(ID) + "," + str(brandname) + "," + str(topic) + "," + \
        str(topicname) + "," + str(twittertext) + "," + str(facebooktext)+'\n'
    facebooktext = ''
    twittertext = ''

# Open and write in the CSV File
with open(archivo_sent, 'w', encoding='UTF-8') as file:
    file.write(texto)